In [2]:
import os
import pyart
from matplotlib import pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import xarray as xr
from datetime import datetime, timedelta

from metpy import plots, calc
from metpy.units import units
import profile_io as io

import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [6]:
# for each 100 second interval, plot the trajectory of the hailsonde ontop of a 4 panel figure of the nearest xsection in space and time.

output_folder = '/g/data/kl02/jss548/hail-research/hailsonde/nhp_output'

#config for dataset
dataset1_config = {'split':271,'sondeid':'9931','timestamp':'2023-07-24_1631'}
dataset2_config = {'split':220,'sondeid':'9935','timestamp':'2023-07-24_1636'}

#set dataset
dataset_config = dataset2_config

#read data
row_to_split_profile = dataset_config['split']
timestamp = dataset_config['timestamp']
sonde_id = dataset_config['sondeid']
filename = f'/g/data/kl02/jss548/hail-research/hailsonde/20230724_hailsonde/{timestamp}_{sonde_id}.raw_flight_history.csv'
with_balloon_profile, no_balloon_profile, metadata = io.decode_raw_flight_history(filename, split=row_to_split_profile, remove_nan_rows=True)

gmt_offset = -6
snd_start_dt = datetime.strptime(os.path.basename(filename)[0:15], '%Y-%m-%d_%H%M') - timedelta(hours = gmt_offset)

#set interval
plot_interval = 100
plot_timesteps = np.arange(plot_interval,no_balloon_profile['time'][-1],plot_interval)

#load radars
radar_site_alt = 810 #m
radar_vols_path = '/g/data/kl02/jss548/hail-research/hailsonde/20230724_CASCV_subset'
radars, radar_file_dt, radar_midpoint_dt_list = io.load_nhp_radar_data(radar_vols_path, 360)

In [7]:
for timestep in plot_timesteps:
    #
    print(timestep)
    #set target time
    target_dt = snd_start_dt + timedelta(seconds = timestep)
    #find nearest timestep and extract profile data
    if timestep <= with_balloon_profile['time'][-1]:
        #ends within "with balloon" profile
        idx = io.find_nearest_idx(with_balloon_profile['time'], timestep)
        with_balloon_profile_subset = {}
        for field in with_balloon_profile:
            with_balloon_profile_subset[field] = with_balloon_profile[field][0:idx]
        no_balloon_profile_subset = None
        target_lat = with_balloon_profile_subset['lat'][-1]
        target_lon = with_balloon_profile_subset['lon'][-1]
        target_alt = with_balloon_profile_subset['hght']._magnitude[-1]
    else:
        #ends within "no balloon" profile
        with_balloon_profile_subset = with_balloon_profile
        idx = io.find_nearest_idx(no_balloon_profile['time'], timestep)
        no_balloon_profile_subset = {}
        for field in no_balloon_profile:
            no_balloon_profile_subset[field] = no_balloon_profile[field][0:idx]
        target_lat = no_balloon_profile_subset['lat'][-1]
        target_lon = no_balloon_profile_subset['lon'][-1]
        target_alt = no_balloon_profile_subset['hght']._magnitude[-1]
    #find nearest radar volume
    radar_idx = io.find_nearest_dt_idx(target_dt, radar_midpoint_dt_list)
    
    #find nearest ppi sweep in volume
    gate_lon = radars[radar_idx].gate_longitude['data']
    gate_lat = radars[radar_idx].gate_latitude['data']
    gate_alt = radars[radar_idx].gate_altitude['data'] + radar_site_alt
    ray_time = radars[radar_idx].time['data']
    ray_elevation = radars[radar_idx].elevation['data']
    dist = np.sqrt((io.degrees2meters(gate_lat - target_lat))**2 + (io.degrees2meters(gate_lon - target_lon))**2 + (gate_alt - target_alt)**2)
    min_dist_idx = np.unravel_index(np.abs(dist).argmin(), gate_lon.shape)
    sweep_idx = io.find_nearest_idx(radars[radar_idx].fixed_angle['data'], ray_elevation[min_dist_idx[0]])

    #calculate radar data point time
    radar_sample_elv = int(ray_elevation[min_dist_idx[0]]*10)/10
    radar_sample_alt = round(gate_alt[min_dist_idx])
    radar_sample_dt = datetime.strptime(radars[radar_idx].time['units'][14:], '%Y-%m-%dT%H:%M:%SZ') + timedelta(seconds=round(ray_time[min_dist_idx[0]]))
    radar_vol_scan0_dt = datetime.strptime(radars[radar_idx].time['units'][14:], '%Y-%m-%dT%H:%M:%SZ')
    radar_vol_fn_dt = radar_file_dt[radar_idx]

    #plot
    fig = plt.figure(figsize=(20, 10))
    min_lon=-116.35
    max_lon=-116.0
    min_lat=53.3
    max_lat=53.6
    storm_u = 8.15
    storm_v = 1.42
    #correct phidp
    display = pyart.graph.RadarMapDisplay(radars[radar_idx])
    
    ax = fig.add_subplot(231, projection=ccrs.PlateCarree())
    display.plot_ppi_map('DBZH', 
                            sweep=sweep_idx, ax=ax, vmin=-10, vmax=70,
                            lon_lines=np.arange(-117, -115, 0.05),
                            lat_lines=np.arange(53, 54, 0.05),
                            min_lon=min_lon,
                            max_lon=max_lon,
                            min_lat=min_lat,
                            max_lat=max_lat)
    display.plot_line_geo(with_balloon_profile_subset['lon'], with_balloon_profile_subset['lat'], 'r-')    
    if no_balloon_profile_subset is not None:
        display.plot_line_geo(no_balloon_profile_subset['lon'], no_balloon_profile_subset['lat'], 'b-')
    display.plot_point(target_lon,target_lat,symbol='ko')

    ax = fig.add_subplot(232, projection=ccrs.PlateCarree())
    display.plot_ppi_map('ZDR', 
                            sweep=sweep_idx, ax=ax, vmin=-2, vmax=5, cmap='pyart_RefDiff',
                            lon_lines=np.arange(-117, -115, 0.05),
                            lat_lines=np.arange(53, 54, 0.05),
                            min_lon=min_lon,
                            max_lon=max_lon,
                            min_lat=min_lat,
                            max_lat=max_lat)
    display.plot_line_geo(with_balloon_profile_subset['lon'], with_balloon_profile_subset['lat'], 'r-')    
    if no_balloon_profile_subset is not None:
        display.plot_line_geo(no_balloon_profile_subset['lon'], no_balloon_profile_subset['lat'], 'b-')
    display.plot_point(target_lon,target_lat,symbol='ko')

    ax = fig.add_subplot(233, projection=ccrs.PlateCarree())
    display.plot_ppi_map('RHOHV', 
                            sweep=sweep_idx, ax=ax, vmin=0.9, vmax=1, cmap='pyart_RefDiff',
                            lon_lines=np.arange(-117, -115, 0.05),
                            lat_lines=np.arange(53, 54, 0.05),
                            min_lon=min_lon,
                            max_lon=max_lon,
                            min_lat=min_lat,
                            max_lat=max_lat)
    display.plot_line_geo(with_balloon_profile_subset['lon'], with_balloon_profile_subset['lat'], 'r-')    
    if no_balloon_profile_subset is not None:
        display.plot_line_geo(no_balloon_profile_subset['lon'], no_balloon_profile_subset['lat'], 'b-')
    display.plot_point(target_lon,target_lat,symbol='ko')

    ax = fig.add_subplot(234, projection=ccrs.PlateCarree())
    display.plot_ppi_map('KDP', 
                            sweep=sweep_idx, ax=ax, vmin=-0.5, vmax=1.5, cmap='pyart_Theodore16',
                            lon_lines=np.arange(-117, -115, 0.05),
                            lat_lines=np.arange(53, 54, 0.05),
                            min_lon=min_lon,
                            max_lon=max_lon,
                            min_lat=min_lat,
                            max_lat=max_lat)
    display.plot_line_geo(with_balloon_profile_subset['lon'], with_balloon_profile_subset['lat'], 'r-')    
    if no_balloon_profile_subset is not None:
        display.plot_line_geo(no_balloon_profile_subset['lon'], no_balloon_profile_subset['lat'], 'b-')
    display.plot_point(target_lon,target_lat,symbol='ko')

    ax = fig.add_subplot(235, projection=ccrs.PlateCarree())
    winds_ffn = f'/g/data/kl02/jss548/hail-research/hailsonde/20230724_3Dwinds/1_{radar_vol_fn_dt.strftime("%Y%m%d_%H%M")}.nc'
    with xr.open_dataset(winds_ffn) as ds:
        #find index of nearest level
        z = np.array(ds['z'][:])
        level_idx = np.argmin(np.abs(z-radar_sample_alt))
        #extract data
        reflectivity = np.array(ds['dBZ_radar1'][level_idx,:,:])
    x = np.linspace(-200, 200, 201)*1000
    x_grid, y_grid = np.meshgrid(x, x)

    lon_0 = radars[radar_idx].longitude['data'][0]
    lat_0 = radars[radar_idx].latitude['data'][0]
    lon, lat = pyart.core.cartesian_to_geographic_aeqd(x_grid, y_grid, lon_0, lat_0, R=6370997.0)


    pc = ax.pcolormesh(lon, lat, reflectivity, vmin=-10, vmax=70, cmap='pyart_HomeyerRainbow', transform=ccrs.PlateCarree())
    ax.set_xlim([min_lon, max_lon])
    ax.set_ylim([min_lat, max_lat])
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='gray', alpha=0.5, linestyle='-')
    gl.top_labels = False
    gl.right_labels = False
    gl.xlocator = mticker.FixedLocator(np.arange(-117, -115, 0.05))
    gl.ylocator = mticker.FixedLocator(np.arange(53, 54, 0.05))
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    ax.set_title(f'refl at {z[level_idx]}m from {radar_vol_fn_dt.strftime("%H:%M")}.nc')
    fig.colorbar(pc, location='right', label='dBZ')
    ax.plot(with_balloon_profile_subset['lon'], with_balloon_profile_subset['lat'], linestyle='-', color='r', transform=ccrs.PlateCarree())    
    if no_balloon_profile_subset is not None:
        ax.plot(no_balloon_profile_subset['lon'], no_balloon_profile_subset['lat'], linestyle='-', color='b', transform=ccrs.PlateCarree())
    ax.plot(target_lon, target_lat, marker='o', color='k', transform=ccrs.PlateCarree())


    # display.plot_ppi_map('UPHIDP', 
    #                         sweep=sweep_idx, ax=ax, cmap='pyart_Wild25',
    #                         lon_lines=np.arange(-117, -115, 0.05), vmin=20, vmax=40,
    #                         lat_lines=np.arange(53, 54, 0.05),
    #                         min_lon=min_lon,
    #                         max_lon=max_lon,
    #                         min_lat=min_lat,
    #                         max_lat=max_lat)
    # display.plot_line_geo(with_balloon_profile_subset['lon'], with_balloon_profile_subset['lat'], 'r-')    
    # if no_balloon_profile_subset is not None:
    #     display.plot_line_geo(no_balloon_profile_subset['lon'], no_balloon_profile_subset['lat'], 'b-')
    # display.plot_point(target_lon,target_lat,symbol='ko')

    ax = fig.add_subplot(236, projection=ccrs.PlateCarree())
    winds_ffn = f'/g/data/kl02/jss548/hail-research/hailsonde/20230724_3Dwinds/1_{radar_vol_fn_dt.strftime("%Y%m%d_%H%M")}.nc'
    with xr.open_dataset(winds_ffn) as ds:
        #find index of nearest level
        z = np.array(ds['z'][:])
        level_idx = np.argmin(np.abs(z-radar_sample_alt))
        #extract data
        vx = np.array(ds['vx'][level_idx,:,:])
        vy = np.array(ds['vy'][level_idx,:,:])
        vz = np.array(ds['vz'][level_idx,:,:])
    x = np.linspace(-200, 200, 201)*1000
    x_grid, y_grid = np.meshgrid(x, x)

    lon_0 = radars[radar_idx].longitude['data'][0]
    lat_0 = radars[radar_idx].latitude['data'][0]
    lon, lat = pyart.core.cartesian_to_geographic_aeqd(x_grid, y_grid, lon_0, lat_0, R=6370997.0)

    #replace -999 with nan
    vx[vx==-999] = np.nan
    vy[vy==-999] = np.nan
    vz[vz==-999] = np.nan
    #apply storm motion offset
    vx = vx - storm_u #pos vx is north
    vy = vy - storm_v #pos vy is east

    pc = ax.pcolormesh(lon, lat, vz, vmin=-5, vmax=5, cmap='RdBu_r', transform=ccrs.PlateCarree())
    q = ax.quiver(lon, lat, vx, vy, transform=ccrs.PlateCarree(), scale=100)
    ax.quiverkey(q, 0.9, 0.1, 5, '5 m/s')
    ax.set_xlim([min_lon, max_lon])
    ax.set_ylim([min_lat, max_lat])
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='gray', alpha=0.5, linestyle='-')
    gl.top_labels = False
    gl.right_labels = False
    gl.xlocator = mticker.FixedLocator(np.arange(-117, -115, 0.05))
    gl.ylocator = mticker.FixedLocator(np.arange(53, 54, 0.05))
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    ax.set_title(f's.r. winds at {z[level_idx]}m from {radar_vol_fn_dt.strftime("%H:%M")}.nc')
    fig.colorbar(pc, location='right', label='w (m/s)')
    ax.plot(with_balloon_profile_subset['lon'], with_balloon_profile_subset['lat'], linestyle='-', color='r', transform=ccrs.PlateCarree())    
    if no_balloon_profile_subset is not None:
        ax.plot(no_balloon_profile_subset['lon'], no_balloon_profile_subset['lat'], linestyle='-', color='b', transform=ccrs.PlateCarree())
    ax.plot(target_lon, target_lat, marker='o', color='k', transform=ccrs.PlateCarree())


    plt.suptitle(f'{radar_sample_elv}deg PPI for t={int(timestep)}s.\n' + 
                 f'sonde sample at {target_alt}m {target_dt.strftime("%H:%M:%S")} \n' +
                  f'radar sample at {radar_sample_alt}m {radar_sample_dt.strftime("%H:%M:%S")} from {radar_vol_fn_dt.strftime("%H:%M")}',
                  fontsize=14)

    plt.savefig(f'{output_folder}/{sonde_id}_ppi_at_{int(timestep)}s.png')
    plt.close()



100.0


/g/data/en0/jss548/miniconda3/envs/hailsonde/lib/python3.11/site-packages/xarray/coding/variables.py:148: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  condition |= data == fv
/g/data/en0/jss548/miniconda3/envs/hailsonde/lib/python3.11/site-packages/xarray/coding/variables.py:148: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  condition |= data == fv


200.0


/g/data/en0/jss548/miniconda3/envs/hailsonde/lib/python3.11/site-packages/xarray/coding/variables.py:148: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  condition |= data == fv
/g/data/en0/jss548/miniconda3/envs/hailsonde/lib/python3.11/site-packages/xarray/coding/variables.py:148: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  condition |= data == fv


300.0


...

/g/data/en0/jss548/miniconda3/envs/hailsonde/lib/python3.11/site-packages/xarray/coding/variables.py:148: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  condition |= data == fv
/g/data/en0/jss548/miniconda3/envs/hailsonde/lib/python3.11/site-packages/xarray/coding/variables.py:148: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  condition |= data == fv


600.0


/g/data/en0/jss548/miniconda3/envs/hailsonde/lib/python3.11/site-packages/xarray/coding/variables.py:148: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  condition |= data == fv
/g/data/en0/jss548/miniconda3/envs/hailsonde/lib/python3.11/site-packages/xarray/coding/variables.py:148: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  condition |= data == fv


700.0


/g/data/en0/jss548/miniconda3/envs/hailsonde/lib/python3.11/site-packages/xarray/coding/variables.py:148: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  condition |= data == fv
/g/data/en0/jss548/miniconda3/envs/hailsonde/lib/python3.11/site-packages/xarray/coding/variables.py:148: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  condition |= data == fv
